# Minor Product Polynomial Construction

This notebook has some useful functions for playing around with using ideas from matrix completion to construct MDP convolutional codes. The following cells contain some methods for creating the sliding truncated generator matrices and their associated _minor product polynomials_.

In [1]:
from minor_product_polynomial import *

## Minor product polynomial

The next example shows how to compute the minor product polynomial for a very simple case.

In [2]:
n, k = 3, 2
K = GF(5)

G0 = vandermonde_matrix(K, k, n)
P = minor_product_polynomial(G0)

print(P)

-2*x00^2*x10 - 2*x00*x10^2 - x10^3


## Degree of minor product polynomial

With the below cell we can verify that the degree of the minor product polynomial is the same for the dual code dimension.

In [3]:
for n in range(1, 20):
    for k in range(n + 1):
        assert minor_product_polynomial_degree(n, k) == minor_product_polynomial_degree(n, n - k)
        assert minor_product_polynomial_individual_degree_standard_form(n, k) == minor_product_polynomial_individual_degree_standard_form(n, n - k)
        
print("The (individual) degree of the minor product polynomial is the same for the dual dimension")

The (individual) degree of the minor product polynomial is the same for the dual dimension


## Minors are nonzero

The following cells show that the minor polynomials are nonzero by choosing a suitable evaluation.

In [4]:
# create one of the 2k x 2k submatrices, denoted by M, and compute its determinant
n, k = 6, 3
K = GF(17)

G0 = cauchy_matrix(K, k, n)

G = create_generator_matrix(G0)
M = G[:, [0, 6, 7, 8, 9, 10]]
M.base_ring().inject_variables()
print(M)
print(M.det())

Defining x00, x01, x02, x10, x11, x12, x20, x21, x22
[  1 x00 x01 x02   0   0]
[ -8 x10 x11 x12   0   0]
[  6 x20 x21 x22   0   0]
[  0   1  -8   6  -4   7]
[  0  -8   6  -4   7   3]
[  0   6  -4   7   3   5]
-8*x01*x10 - 8*x02*x10 + 8*x00*x11 - 4*x02*x11 + 8*x00*x12 + 4*x01*x12 - 5*x01*x20 - 5*x02*x20 - 7*x11*x20 - 7*x12*x20 + 5*x00*x21 + 6*x02*x21 + 7*x10*x21 + 5*x12*x21 + 5*x00*x22 - 6*x01*x22 + 7*x10*x22 - 5*x11*x22


In [5]:
# the following evaluation shows that the polynomial M.det() is nonzero
M_eval = M.subs({x00: 1, x01: 0, x02: 0, x10: 0, x11: 1, x12: 0, x20: 0, x21: 0, x22: 0}) # type: ignore
print(M_eval)
print(M_eval.det())

[ 1  1  0  0  0  0]
[-8  0  1  0  0  0]
[ 6  0  0  0  0  0]
[ 0  1 -8  6 -4  7]
[ 0 -8  6 -4  7  3]
[ 0  6 -4  7  3  5]
8


## Required field size

The next cells show how large the field size needs to be for some small parameters.

In [6]:
print("[n, k] : degree of polynomial -> individual degree if G0 is in standard form")
print()

for n in range(2, 10):
    for k in range(1, n):
        print(f"[{n}, {k}] :",
              minor_product_polynomial_degree(n, k),
              "->",
              minor_product_polynomial_individual_degree_standard_form(n, k),
             )
    print()

[n, k] : degree of polynomial -> individual degree if G0 is in standard form

[2, 1] : 1 -> 1

[3, 1] : 3 -> 2
[3, 2] : 3 -> 2

[4, 1] : 6 -> 3
[4, 2] : 18 -> 10
[4, 3] : 6 -> 3

[5, 1] : 10 -> 4
[5, 2] : 60 -> 28
[5, 3] : 60 -> 28
[5, 4] : 10 -> 4

[6, 1] : 15 -> 5
[6, 2] : 150 -> 60
[6, 3] : 300 -> 126
[6, 4] : 150 -> 60
[6, 5] : 15 -> 5

[7, 1] : 21 -> 6
[7, 2] : 315 -> 110
[7, 3] : 1050 -> 396
[7, 4] : 1050 -> 396
[7, 5] : 315 -> 110
[7, 6] : 21 -> 6

[8, 1] : 28 -> 7
[8, 2] : 588 -> 182
[8, 3] : 2940 -> 1001
[8, 4] : 4900 -> 1716
[8, 5] : 2940 -> 1001
[8, 6] : 588 -> 182
[8, 7] : 28 -> 7

[9, 1] : 36 -> 8
[9, 2] : 1008 -> 280
[9, 3] : 7056 -> 2184
[9, 4] : 17640 -> 5720
[9, 5] : 17640 -> 5720
[9, 6] : 7056 -> 2184
[9, 7] : 1008 -> 280
[9, 8] : 36 -> 8



In [7]:
# this is the smallest field we have at the moment
n, k = 5, 3
q = 9
K = GF(q)

G0 = cauchy_matrix(K, k, n).rref()

G = find_solution(G0)

print(G)

[       1        0        0        1        2|      z2   z2 + 1        0        0        0]
[       0        1        0     2*z2        1|      z2        2        0        0        0]
[       0        0        1 2*z2 + 2 2*z2 + 1|      z2 2*z2 + 1        0        0        0]
[--------------------------------------------+--------------------------------------------]
[       0        0        0        0        0|       1        0        0        1        2]
[       0        0        0        0        0|       0        1        0     2*z2        1]
[       0        0        0        0        0|       0        0        1 2*z2 + 2 2*z2 + 1]
